In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
df = pd.read_csv('dataset (1).csv')

# Initial preprocessing
df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['VehicleTypeWeight'] = df['VehicleType'].map({'car': 1.0, 'bike': 0.6, 'truck': 1.5})
df['TrafficLevel'] = df['TrafficConditionNearby'].map({'low': 0.3, 'medium': 0.6, 'high': 1.0})
df = df.sort_values(by=['SystemCodeNumber', 'Datetime']).reset_index(drop=True)
df['BasePrice'] = 10.0

In [4]:
def baseline_pricing(df, alpha=2.0):
    prices = []
    prev_prices = {}

    for i, row in df.iterrows():
        lot_id = row['SystemCodeNumber']
        if lot_id not in prev_prices:
            prev_prices[lot_id] = row['BasePrice']

        price = prev_prices[lot_id] + alpha * row['OccupancyRate']
        price = max(5, min(20, price))  # price bound between $5 and $20
        prev_prices[lot_id] = price
        prices.append(price)

    df['Price_Model1'] = prices
    return df

df = baseline_pricing(df)
df[['SystemCodeNumber', 'Datetime', 'OccupancyRate', 'Price_Model1']].head()


,SystemCodeNumber,Datetime,OccupancyRate,Price_Model1
0,BHMBCCMKT01,2016-10-04 07:59:00,0.105719,10.211438
1,BHMBCCMKT01,2016-10-04 08:25:00,0.110919,10.433276
2,BHMBCCMKT01,2016-10-04 08:59:00,0.138648,10.710572
3,BHMBCCMKT01,2016-10-04 09:32:00,0.185442,11.081456
4,BHMBCCMKT01,2016-10-04 09:59:00,0.259965,11.601386


In [5]:
def demand_based_pricing(df, alpha=1.5, beta=0.7, gamma=0.5, delta=0.8, epsilon=0.5, lambd=0.8):
    norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)) if np.max(x) != np.min(x) else x
    df['Demand'] = (alpha * df['OccupancyRate'] + beta * df['QueueLength']
                    - gamma * df['TrafficLevel'] + delta * df['IsSpecialDay']
                    + epsilon * df['VehicleTypeWeight'])
    df['Demand_Norm'] = norm(df['Demand'])
    df['Price_Model2'] = df['BasePrice'] * (1 + lambd * df['Demand_Norm'])
    df['Price_Model2'] = df['Price_Model2'].clip(lower=5, upper=20)  # bound
    return df

df = demand_based_pricing(df)
df[['SystemCodeNumber', 'Datetime', 'Demand_Norm', 'Price_Model2']].head()


,SystemCodeNumber,Datetime,Demand_Norm,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,0.082697,10.661575
1,BHMBCCMKT01,2016-10-04 08:25:00,0.083323,10.666585
2,BHMBCCMKT01,2016-10-04 08:59:00,0.142865,11.142924
3,BHMBCCMKT01,2016-10-04 09:32:00,0.148501,11.188008
4,BHMBCCMKT01,2016-10-04 09:59:00,0.141418,11.131346


In [6]:
import math
# Haversine distance to calculate proximity between parking lots
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Compute competitors for each lot
def get_competitor_prices(row, df, radius_km=1.0):
    lot_id = row['SystemCodeNumber']
    lat, lon = row['Latitude'], row['Longitude']
    time = row['Datetime']

    nearby_lots = df[(df['Datetime'] == time) & (df['SystemCodeNumber'] != lot_id)].copy()
    nearby_lots['Distance'] = nearby_lots.apply(
        lambda r: haversine(lat, lon, r['Latitude'], r['Longitude']), axis=1)

    competitors = nearby_lots[nearby_lots['Distance'] <= radius_km]
    if competitors.empty:
        return row['Price_Model2']

    avg_comp_price = competitors['Price_Model2'].mean()
    if row['Occupancy'] >= row['Capacity']:
        return min(row['Price_Model2'], avg_comp_price)  # reduce price if full and others are cheaper
    elif avg_comp_price > row['Price_Model2']:
        return min(row['Price_Model2'] + 1.0, 20)
    return row['Price_Model2']

df['Price_Model3'] = df.apply(lambda row: get_competitor_prices(row, df), axis=1)
df[['SystemCodeNumber', 'Datetime', 'Price_Model2', 'Price_Model3']].head()


,SystemCodeNumber,Datetime,Price_Model2,Price_Model3
0,BHMBCCMKT01,2016-10-04 07:59:00,10.661575,11.661575
1,BHMBCCMKT01,2016-10-04 08:25:00,10.666585,11.666585
2,BHMBCCMKT01,2016-10-04 08:59:00,11.142924,12.142924
3,BHMBCCMKT01,2016-10-04 09:32:00,11.188008,12.188008
4,BHMBCCMKT01,2016-10-04 09:59:00,11.131346,12.131346


In [7]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.io import push_notebook
import pandas as pd

output_notebook()

# Ensure datetime
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Initial lot
lot_ids = df['SystemCodeNumber'].unique().tolist()
initial_lot = lot_ids[0]

def get_filtered_data(lot_id):
    lot_df = df[df['SystemCodeNumber'] == lot_id].sort_values('Datetime')
    return ColumnDataSource(data=dict(
        Datetime=lot_df['Datetime'],
        Model1=lot_df['Price_Model1'],
        Model2=lot_df['Price_Model2'],
        Model3=lot_df['Price_Model3']
    ))

source = get_filtered_data(initial_lot)

# Create the figure
p = figure(title=f"Pricing Trends - {initial_lot}", x_axis_type='datetime', width=900, height=400)
line1 = p.line(x='Datetime', y='Model1', source=source, color='blue', legend_label='Model 1 (Linear)')
line2 = p.line(x='Datetime', y='Model2', source=source, color='green', legend_label='Model 2 (Demand)')
line3 = p.line(x='Datetime', y='Model3', source=source, color='red', legend_label='Model 3 (Competitive)')

p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"

# Create dropdown for lot selection
dropdown = Select(title="Select Parking Lot", value=initial_lot, options=lot_ids)

# Define callback (in Python, not JS)
def update_plot(attr, old, new):
    new_source = get_filtered_data(new)
    source.data = new_source.data
    p.title.text = f"Pricing Trends - {new}"

dropdown.on_change('value', update_plot)

# Display
show(column(dropdown, p))


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

